In [126]:
import numpy as np

**Module** is an abstract class which defines fundamental methods necessary for a training a neural network. You do not need to change anything here, just read the comments.

In [127]:
class Module(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        self.training = True
    """
    Basically, you can think of a module as of a something (black box) 
    which can process `input` data and produce `ouput` data.
    This is like applying a function which is called `forward`: 
        
        output = module.forward(input)
    
    The module should be able to perform a backward pass: to differentiate the
    `forward` function. 
    More, it should be able to differentiate it if is a part of chain (chain rule).
    The latter implies there is a gradient from previous step of a chain rule. 
    
        gradInput = module.backward(input, gradOutput)
    """
    
    def forward(self, input):
        """
        Takes an input object, and computes the corresponding output of the module.
        """
        return self.updateOutput(input)

    def backward(self, input, gradOutput):
        """
        Performs a backpropagation step through the module, with respect to the given input.
        
        This includes 
         - computing a gradient w.r.t. `input` (is needed for further backprop),
         - computing a gradient w.r.t. parameters (to update parameters while optimizing).
        """
        self.updateGradInput(input, gradOutput)
        self.accGradParameters(input, gradOutput)
        return self.gradInput
    

    def updateOutput(self, input):
        """
        Computes the output using the current parameter set of the class and input.
        This function returns the result which is stored in the `output` field.
        
        Make sure to both store the data in `output` field and return it. 
        """
        
        # The easiest case:
            
        # self.output = input 
        # return self.output
        
        pass

    def updateGradInput(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own input. 
        This is returned in `gradInput`. Also, the `gradInput` state variable
        is updated accordingly.
        
        The shape of `gradInput` is always the same as the shape of `input`.
        
        Make sure to both store the gradients in `gradInput` field and return it.
        """
        
        # The easiest case:
        
        # self.gradInput = gradOutput 
        # return self.gradInput
        
        pass   
    
    def accGradParameters(self, input, gradOutput):
        """
        Computing the gradient of the module with respect to its own parameters.
        No need to override if module has no parameters (e.g. ReLU).
        """
        pass
    
    def zeroGradParameters(self): 
        """
        Zeroes `gradParams` variable if the module has params.
        """
        pass
        
    def getParameters(self):
        """
        Returns a list with its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
        
    def getGradParameters(self):
        """
        Returns a list with gradients with respect to its parameters. 
        If the module does not have parameters return empty list. 
        """
        return []
    
    def training(self):
        """
        Sets training mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = True
    
    def evaluate(self):
        """
        Sets evaluation mode for the module.
        Training and testing behaviour differs for Dropout, BatchNorm.
        """
        self.training = False
    
    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Module"

# Sequential container

**Define** a forward and backward pass procedures.

In [146]:
class Sequential(Module):
    """
         This class implements a container, which processes `input` data sequentially. 
         
         `input` is processed by each module (layer) in self.modules consecutively.
         The resulting array is called `output`. 
    """
    
    def __init__ (self):
        super(Sequential, self).__init__()
        self.modules = []
   
    def add(self, module):
        """
        Adds a module to the container.
        """
        self.modules.append(module)

    def updateOutput(self, input):
        """
        Basic workflow of FORWARD PASS:
        
            y_0    = module[0].forward(input)
            y_1    = module[1].forward(y_0)
            ...
            output = module[n-1].forward(y_{n-2})   
            
            
        Just write a little loop. 
        """

        self.output = input
        for module in self.modules:
            self.output = module.forward(self.output)
            module.output = self.output
        return self.output

    def backward(self, input, gradOutput):
        """
        Workflow of BACKWARD PASS:
            
            g_{n-1} = module[n-1].backward(y_{n-2}, gradOutput)
            g_{n-2} = module[n-2].backward(y_{n-3}, g_{n-1})
            ...
            g_1 = module[1].backward(y_0, g_2)   
            gradInput = module[0].backward(input, g_1)   
             
             
        !!!
                
        To ech module you need to provide the input, module saw while forward pass, 
        it is used while computing gradients. 
        >>>updated line
        Make sure that the input for `i-th` layer the output of `module[i - 1]`
        (just the same input as in forward pass) 
        and NOT `input` to this Sequential module. 
        
        !!!
        
        """
        g = gradOutput
        for i, module in enumerate(reversed(self.modules)):
            if i < len(self.modules) - 1:
                y = self.modules[len(self.modules) - 1 - (i + 1)].output
            else:
                y = input
            g = module.backward(y, g)
        self.gradInput = g
        return self.gradInput
      

    def zeroGradParameters(self): 
        for module in self.modules:
            module.zeroGradParameters()
    
    def getParameters(self):
        """
        Should gather all parameters in a list.
        """
        return [x.getParameters() for x in self.modules]
    
    def getGradParameters(self):
        """
        Should gather all gradients w.r.t parameters in a list.
        """
        return [x.getGradParameters() for x in self.modules]
    
    def __repr__(self):
        string = "".join([str(x) + '\n' for x in self.modules])
        return string
    
    def __getitem__(self,x):
        return self.modules.__getitem__(x)

# Layers

- input:   **`batch_size x n_feats1`**
- output: **`batch_size x n_feats2`**

In [129]:
class Linear(Module):
    """
    A module which applies a linear transformation 
    A common name is fully-connected layer, InnerProductLayer in caffe. 
    
    The module should work with 2D input of shape (n_samples, n_feature).
    """
    def __init__(self, n_in, n_out):
        super(Linear, self).__init__()
       
        # This is a nice initialization
        stdv = 1./np.sqrt(n_in)
        self.W = np.random.uniform(-stdv, stdv, size = (n_out, n_in))
        self.b = np.random.uniform(-stdv, stdv, size = n_out)
        
        self.gradW = np.zeros_like(self.W)
        self.gradb = np.zeros_like(self.b)
        
    def updateOutput(self, input):
        self.output = np.dot(input, np.transpose(self.W)) + self.b
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.dot(gradOutput, self.W)
        return self.gradInput
    
    def accGradParameters(self, input, gradOutput):
        self.gradW = np.dot(np.transpose(gradOutput), input)
        self.gradb = np.sum(gradOutput, axis = 0)
        pass
    
    def zeroGradParameters(self):
        self.gradW.fill(0)
        self.gradb.fill(0)
        
    def getParameters(self):
        return [self.W, self.b]
    
    def getGradParameters(self):
        return [self.gradW, self.gradb]
    
    def __repr__(self):
        s = self.W.shape
        q = 'Linear %d -> %d' %(s[1],s[0])
        return q

This one is probably the hardest but as others only takes 5 lines of code in total. 
- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [144]:
class SoftMax(Module):
    def __init__(self):
         super(SoftMax, self).__init__()
    
    def updateOutput(self, input):
        # start with normalization for numerical stability
        input = np.subtract(input, input.max(axis=1, keepdims=True))
        
        probs = np.exp(input)
        probs /= np.sum(probs, axis=1, keepdims=True)
        self.output = probs
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        input = np.subtract(input, input.max(axis=1, keepdims=True))
        
        self.gradInput = np.zeros(shape=input.shape)
        not_normed_probs = np.exp(np.subtract(input, input.max(axis=1, keepdims=True)))
        normed_probs = not_normed_probs / np.sum(not_normed_probs, axis=1, keepdims=True)
        over_normed_probs = not_normed_probs / np.sum(not_normed_probs, axis=1, keepdims=True) ** 2
        for i in range(input.shape[0]):
            almost_grad = -over_normed_probs[i].reshape(-1, 1).dot(not_normed_probs[i].reshape(1, -1))
            almost_grad += np.diag(normed_probs[i])
            self.gradInput[i] = gradOutput[i].dot(almost_grad)
        
        return self.gradInput
    
    def __repr__(self):
        return "SoftMax"

Implement [**dropout**](https://www.cs.toronto.edu/~hinton/absps/JMLRdropout.pdf). The idea and implementation is really simple: just multimply the input by $Bernoulli(p)$ mask. 

This is a very cool regularizer. In fact, when you see your net is overfitting try to add more dropout.

While training (`self.training == True`) it should sample a mask on each iteration (for every batch). When testing this module should implement identity transform i.e. `self.output = input`.

- input:   **`batch_size x n_feats`**
- output: **`batch_size x n_feats`**

In [131]:
class Dropout(Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        
        self.p = p
        self.mask = None
        
    def updateOutput(self, input):
        self.mask = np.random.binomial(size=input.shape, p=self.p, n=1)
        self.output = np.multiply(input, self.mask)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        self.mask = np.random.binomial(size=gradOutput.shape, p=self.p, n=1)
        self.gradInput = np.multiply(gradOutput, self.mask)
        return self.gradInput
        
    def __repr__(self):
        return "Dropout"

# Activation functions

Here's the complete example for the **Rectified Linear Unit** non-linearity (aka **ReLU**): 

In [132]:
class ReLU(Module):
    def __init__(self):
         super(ReLU, self).__init__()
    
    def updateOutput(self, input):
        self.output = np.maximum(input, 0)
        return self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0)
        return self.gradInput
    
    def __repr__(self):
        return "ReLU"

Implement [**Leaky Rectified Linear Unit**](http://en.wikipedia.org/wiki%2FRectifier_%28neural_networks%29%23Leaky_ReLUs). Expriment with slope. 

In [133]:
class LeakyReLU(Module):
    def __init__(self, slope = 0.03):
        super(LeakyReLU, self).__init__()
            
        self.slope = slope
        
    def updateOutput(self, input):
        self.output = np.maximum(input, input * self.slope)
        return  self.output
    
    def updateGradInput(self, input, gradOutput):
        self.gradInput = np.multiply(gradOutput , input > 0) 
        + np.multiply(gradOutput, input <= 0) * self.slope 
        return self.gradInput
    
    def __repr__(self):
        return "LeakyReLU"

# Criterions

Criterions are used to score the models answers. 

In [134]:
class Criterion(object):
    def __init__ (self):
        self.output = None
        self.gradInput = None
        
    def forward(self, input, target):
        """
            Given an input and a target, compute the loss function 
            associated to the criterion and return the result.
            
            For consistency this function should not be overrided,
            all the code goes in `updateOutput`.
        """
        return self.updateOutput(input, target)

    def backward(self, input, target):
        """
            Given an input and a target, compute the gradients of the loss function
            associated to the criterion and return the result. 

            For consistency this function should not be overrided,
            all the code goes in `updateGradInput`.
        """
        return self.updateGradInput(input, target)
    
    def updateOutput(self, input, target):
        """
        Function to override.
        """
        return self.output

    def updateGradInput(self, input, target):
        """
        Function to override.
        """
        return self.gradInput   

    def __repr__(self):
        """
        Pretty printing. Should be overrided in every module if you want 
        to have readable description. 
        """
        return "Criterion"

The **MSECriterion**, which is basic L2 norm usually used for regression, is implemented here for you.

In [135]:
class MSECriterion(Criterion):
    def __init__(self):
        super(MSECriterion, self).__init__()
        
    def updateOutput(self, input, target):   
        self.output = np.sum(np.power(input - target,2)) / input.shape[0]
        return self.output 
 
    def updateGradInput(self, input, target):
        self.gradInput  = (input - target) * 2 / input.shape[0]
        return self.gradInput

    def __repr__(self):
        return "MSECriterion"

You task is to implement the **ClassNLLCriterion**. It should implement [multiclass log loss](http://scikit-learn.org/stable/modules/model_evaluation.html#log-loss). Nevertheless there is a sum over `y` (target) in that formula, 
remember that targets are one-hot encoded. This fact simplifies the computations a lot. Note, that criterions are the only places, where you divide by batch size. 

In [138]:
class ClassNLLCriterion(Criterion):
    def __init__(self):
        a = super(ClassNLLCriterion, self)
        super(ClassNLLCriterion, self).__init__()
        
    def updateOutput(self, input, target): 
        
        # Use this trick to avoid numerical errors
        eps = 1e-15 
        input_clamp = np.clip(input, eps, 1 - eps)
        
        self.output = -np.sum(np.multiply(target, np.log(input_clamp)), axis=1).mean()
        
        return self.output

    def updateGradInput(self, input, target):
        
        # Use this trick to avoid numerical errors
        input_clamp = np.maximum(1e-15, np.minimum(input, 1 - 1e-15) )
                
        self.gradInput = -np.multiply(target, 1. / input_clamp) / input.shape[0]
        
        return self.gradInput
    
    def __repr__(self):
        return "ClassNLLCriterion"

In [139]:
#checking dimensions

In [140]:
linear = Linear(3, 10)
updateOutput = lambda x: linear.updateOutput(x)
updateGradInput = lambda x, gout: linear.updateGradInput(x, gout)
accGradParameters = lambda x, gout: linear.accGradParameters(x, gout)

In [141]:
updateOutput(np.random.random((11, 3)))
updateGradInput(np.random.random((11, 3)), np.random.random((11, 10)))
accGradParameters(np.random.random((11, 3)), np.random.random((11, 10)))

In [142]:
softMax = SoftMax()
updateOutput = lambda x: softMax.updateOutput(x)
updateGradInput = lambda x, gout: softMax.updateGradInput(x, gout)

In [119]:
updateOutput(np.random.random((11, 3)))
updateGradInput(np.random.random((11, 3)), np.random.random((11, 3)))

array([[-0.05540184,  0.08963212, -0.03423028],
       [ 0.13064925, -0.04791883, -0.08273041],
       [ 0.05231247,  0.11624592, -0.16855839],
       [ 0.12903529, -0.18507261,  0.05603731],
       [ 0.079237  ,  0.10249722, -0.18173421],
       [ 0.01560677,  0.00588073, -0.0214875 ],
       [ 0.11909662, -0.08937473, -0.02972189],
       [ 0.14801218, -0.05160047, -0.09641171],
       [ 0.09287047, -0.16652521,  0.07365475],
       [-0.05247376, -0.10339445,  0.15586821],
       [ 0.01794209, -0.14905043,  0.13110834]])

In [120]:
dropout = Dropout()
updateOutput = lambda x: dropout.updateOutput(x)
updateGradInput = lambda x, gout: dropout.updateGradInput(x, gout)

In [88]:
updateOutput(np.random.random((11, 3)))
updateGradInput(np.random.random((11, 3)), np.random.random((11, 3)))

array([[ 0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.40136443],
       [ 0.66970157,  0.        ,  0.65080473],
       [ 0.76486694,  0.        ,  0.13867965],
       [ 0.48286208,  0.56741432,  0.74273226],
       [ 0.        ,  0.        ,  0.75097807],
       [ 0.54789666,  0.3173787 ,  0.        ],
       [ 0.        ,  0.5934632 ,  0.81635264],
       [ 0.        ,  0.        ,  0.        ],
       [ 0.84880802,  0.00301281,  0.47102175],
       [ 0.23496525,  0.        ,  0.91258508]])

In [93]:
leaky = LeakyReLU()
updateOutput = lambda x: leaky.updateOutput(x)
updateGradInput = lambda x, gout: leaky.updateGradInput(x, gout)

In [94]:
updateOutput(np.random.random((11, 3)))
updateGradInput(np.random.random((11, 3)), np.random.random((11, 3)))

array([[ 0.75547677,  0.50968151,  0.2191078 ],
       [ 0.28440933,  0.98246984,  0.74444765],
       [ 0.32485604,  0.61150318,  0.45643339],
       [ 0.29401337,  0.69610275,  0.75866096],
       [ 0.8495671 ,  0.70230949,  0.05753921],
       [ 0.19109252,  0.84155435,  0.49178458],
       [ 0.30397149,  0.23247517,  0.70179131],
       [ 0.29344314,  0.23844084,  0.73530421],
       [ 0.94666968,  0.4373453 ,  0.30339569],
       [ 0.47745996,  0.77713784,  0.09221389],
       [ 0.35100877,  0.47507351,  0.89162176]])

In [121]:
# y = np.random.randint(0, 3, 10)
# dx = lambda x: softmax_loss(x.reshape((10, 3)), y)[1].reshape(-1)
# loss = lambda x: softmax_loss(x.reshape((10, 3)), y)[0]

In [122]:
# print 'loss is a scalar\n', loss(np.random.random((10, 3)))

In [123]:
# print 'gradient is a matrix with shape 10x3\n', dx(np.random.random((10, 3)))

In [124]:
# print 'difference should be ~10e-8', check_grad(loss, dx, np.random.random((10, 3)).reshape(-1))